# AssociationTable with word embeddings association - association¶

word2vec or Glove? -> Glove because whole corpus relations not only neighboring, useful when dealing with associations?



In [1]:
# creqating tarrget:assoc dataFrame (wit CS) with cosine similarities as values of assoc cosdis compariosn with each other assoc
import pandas as pd
import numpy as np
import gensim.downloader

glove_vectors = gensim.downloader.load('glove-wiki-gigaword-100') # type KeyedVectors<vector_size=100, 400000 keys>

#datatype= 'extreme'
datatype= 'filtered'
#wordclass = 'adj'
wordclass = 'nouns'
#wordclass = 'verbs'

#creating datatable to fill with cosine distances - assoc/targets without embeddings are deleted from dataframe
df = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_'+datatype+'/R1only_types_'+wordclass+'_'+datatype+'_AssocTable.csv')

#df = df.loc[:20,:'significant'] #try slice

targets = list(df['Unnamed: 0'])
assoc = list(df.columns)
cs = df['CS']
assoc.remove('Unnamed: 0')
assoc.remove('CS')

df = df.select_dtypes(include=['number']) != 0  #getting truth values for nonzero
df['target'] = targets  #adding back targets to nonzero truth table

#looping through assoc to find out which ones have embedding
removelist = []
for a in assoc:
    try:
        glove_vectors[a]
    except KeyError:   #if assoc has no embedding
        removelist.append(a)
        continue
assoc = list(set(assoc) - set(removelist)) #remove assoc withut embeddings from assoc list
print('Dropped Associations: ' +str(len(removelist)))

embTable = pd.DataFrame(np.zeros((len(list(df.index)), len( ['Unnamed: 0','CS']+assoc))),index= list(df.index), columns = ['Unnamed: 0','CS']+assoc)
embTable['CS']=cs
embTable['Unnamed: 0'] = targets

#looping through targets and their associations for getting embeddings and calculating cosine distance when in original AssocTable non-zero value 
df =df.drop('CS',axis=1)

for t in targets:
    #print('TARGET: ' + t)
    ind = embTable[embTable['Unnamed: 0'] == t].index[0] #get target inedx
    tassoc = df.iloc[ind].index.values[(df.iloc[ind] == True)]# list of assoc for t
    tassoc = list(set(tassoc) - set(removelist)) #remove assoc withut embeddings from tassoc list
    #print('TASSOC: ' + str(tassoc))
    for a1 in tassoc:
       # print('A1: ' + a1)
        a1cosdis = []
        for a2 in tassoc:    
            #print('A2: ' + a2)
            if a1 != a2: #ruling out a1 == a2 cosdis
                a1cosdis.append(round(glove_vectors.distance(a1,a2),3)) #add a1-a2 cosdis to list
                #print('CosDis of ' +str(a1)+' and '+str(a2))
            else:
               # print('Back')
                continue
        embTable.loc[ind,a1] = np.mean(a1cosdis)   # add average of a1cosdis list to embTable
    
embTable.to_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/M32_CosDisTable/R1only_types_'+wordclass+'_'+datatype+'_CosDisTable32.csv',index=False)
print(embTable)   
            

Dropped Associations: 11257
      Unnamed: 0    CS  sulfuric  drainage  cerebral  causality  midlife  \
0          music  4.31       0.0       0.0       0.0        0.0      0.0   
1            way  2.34       0.0       0.0       0.0        0.0      0.0   
2           none  2.59       0.0       0.0       0.0        0.0      0.0   
3          paper  4.93       0.0       0.0       0.0        0.0      0.0   
4           town  4.64       0.0       0.0       0.0        0.0      0.0   
...          ...   ...       ...       ...       ...        ...      ...   
3693     glamour  2.11       0.0       0.0       0.0        0.0      0.0   
3694  windscreen  4.59       0.0       0.0       0.0        0.0      0.0   
3695       liner  4.20       0.0       0.0       0.0        0.0      0.0   
3696     roadway  4.67       0.0       0.0       0.0        0.0      0.0   
3697       lipid  3.04       0.0       0.0       0.0        0.0      0.0   

      tummy  stylish  dashboard  ...  wrestling  chronologi

In [ ]:
baba = 3602
old = df.iloc[baba].index.values[(df.iloc[baba] == True)]
len(old)

In [ ]:
embTabletruth = embTable.select_dtypes(include=['number']) != 0
new = embTabletruth.iloc[baba].index.values[(embTabletruth.iloc[baba] == True)]
len(new)

In [ ]:
print(len(list(set(old)-set(new))))

In [ ]:
#indext = df[df['Unnamed: 0'] == 'cheap'].index[0]
print(embTable.loc[3601,'kitten'])
print(df.loc[3601,'kitten'])